In [ ]:
import matplotlib
matplotlib.use("PDF")
from pylab import *
from scipy.special import jv, erf
from scipy.integrate import quad

In [ ]:
def Intensity(z):
    return (2.0 * jv(1, z) / z)**2.0

In [ ]:
def ObscuredIntensity(e, z):
    return (2.0 * jv(1, z) / z - 2.0 * e * jv(1, e * z) / z)**2.0 / (1.0 - e**2)**2

In [ ]:
print Intensity(1.616)

In [ ]:
lam = 0.65
N = 20.0
Numvalues = 100
rs = linspace(0.001,10.0,Numvalues)
ivalues = zeros([Numvalues])
for n in range(Numvalues):
    ivalues[n] = Intensity(pi * rs[n] / (lam * N))
print len(rs), len(ivalues)

In [ ]:
plot(rs,ivalues)
show()

In [ ]:
def Convolve(a, z):
    return (erf(z+a) - erf(z-a))/(2.0*erf(a))

In [ ]:
def ConvolvedFull(c, x):
    nx = 100
    xmax = 20.0
    data = linspace(-xmax,xmax,nx)
    sum = 0.0
    for i in range(nx):
        x1 = data[i]
        if abs(x-x1) < c:
            sum += exp(-x1*x1)
    return sum 


In [ ]:
print ConvolvedFull(2.0,-5.0)

In [ ]:
figure()
for a in [0.001,1.0,2.0,3.0,5.0]:
    Numvalues = 100
    rs = linspace(-10.0,10.0,Numvalues)
    ivalues = zeros([Numvalues])
    for n in range(Numvalues):
        ivalues[n] = Convolve(a,rs[n])
    plot(rs,ivalues, label="c=%.2f"%a)
ylim(0,1.1)
xlim(-15,15)
legend(loc='upper left')
savefig("/Users/cslage/Research/LSST/code/GUI/brighter_fatter/20170831_002_spots/Convolved_Gaussians.pdf")
show()
close()

In [ ]:
figure()
for a in [0.1,1.0,2.0,3.0,5.0]:
    Numvalues = 100
    rs = linspace(-10.0,10.0,Numvalues)
    ivalues = zeros([Numvalues])
    ivalues_full = zeros([Numvalues])
    for n in range(Numvalues):
        ivalues[n] = Convolve(a,rs[n])
        ivalues_full[n] = ConvolvedFull(a,rs[n])
    plot(rs,ivalues, label="c=%.2f"%a)
    plot(rs,ivalues_full/ConvolvedFull(a,0.0), ls='--', lw=4.0)
ylim(0,1.1)
xlim(-15,15)
legend(loc='upper left')
show()
close()

In [ ]:
# This is the right form, a convolution of a Gaussian profile with a square "Top Hat"
# Need to work it out in 2D.  If I do this right, it will be just one more parameter
# The numerical convolution works in 1D.

In [ ]:
def CoredGaussian(x, y, sigmax=1.0, sigmay=1.0, r=0.5):
    x2 = x * x
    y2 = y * y
    r2 = r * r
    sigx2 = sigmax * sigmax
    sigy2 = sigmay * sigmay
    sig2 = sigx2 + sigy2
    rx2 = r2 * sigx2 / sig2
    ry2 = r2 * sigy2 / sig2
    rx = sqrt(rx2)
    ry = sqrt(ry2)
    testr = sqrt(x2/rx2 + y2/ry2)
    if testr < 1.0:
    #if x2 + y2 < rx2 + ry2:
         return 1.0
    else:
        return exp(-(testr-1.0)*(testr-1.0)/(2*sigmax*sigmay/rx/ry))
        #return exp(-(x2)/(2*sigx2)) * exp(-(y2)/(2*sigy2)) * exp((r2)/(2*sigx2))

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])

nx = 200
ny = 200
data = Array2d(-5.0,5.0,nx,-5.0,5.0,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = CoredGaussian(data.x[i],data.y[j],sigmax=1.0, sigmay=3.0, r=1.0)
figure()
subplot(1,2,1)
imshow(data.data)
subplot(1,2,2)
plot(data.x,data.data[:,ny/2], color='red')
plot(data.y,data.data[nx/2,:], color='blue')
ylim(0,1.2)
show()

In [ ]:
def CoredGaussian1D(x, sigma=1.0, r=0.5):
    if abs(x) < r:
         return 1.0
    else:
        return exp(-(x-r)*(x-r)/(2*sigma*sigma))

In [ ]:
x = linspace(0.0,5.0,200)
y = zeros([200])
for i,xval in enumerate(x):
    y[i]= CoredGaussian1D(xval, r=0.2)
plot(x,y)
ylim(0,1.2)
show()

In [ ]:
# CoredGaussian doesn't seem to be working.  The functional form looks OK, but
# I don't think it will be analytically integrable.  Returning to the Convolution approach

In [ ]:
def ConvolvedGaussian(x, y, sigmax=1.0, sigmay=1.0, c=0.5):
    ssigx = sigmax * sqrt(2.0)
    ssigy = sigmay * sqrt(2.0)
    sr = sqrt((x/ssigx)**2 + (y/ssigy)**2)
    return (erf(sr+c) - erf(sr-c)) / (erf(c) - erf(-c))

In [ ]:
def ConvolvedGaussianFull(x, y, sigmax=1.0, sigmay=1.0, c=0.5):
    ssigx2 = sigmax * sigmax * 2.0
    ssigy2 = sigmay * sigmay * 2.0
    nx = 100
    ny = 100
    xmax = 10.0
    ymax = 10.0
    data = Array2d(-xmax,xmax,nx,-ymax,ymax,ny)
    sum = 0.0
    c2 = c * c
    for i in range(nx):
        x1 = data.x[i]
        for j in range(ny):
            y1 = data.y[j]
            r2 = (x-x1)*(x-x1) + (y-y1)*(y-y1)
            if r2 < c2:
                sum += exp(-x1*x1/ssigx2) * exp(-y1*y1/ssigy2) * data.dx * data.dy
    return sum 




In [ ]:
print "val = %.9f"%(ConvolvedGaussianFull(0.0,0.0, sigmax = 2.0,sigmay=1.0,c=2.0))
print 2.0 * erf(2.0)

In [ ]:
def Gaussian(x, y, sigmax=1.0, sigmay=1.0):
    ssigx = sigmax * sqrt(2.0)
    ssigy = sigmay * sqrt(2.0)
    sr = sqrt((x/ssigx)**2 + (y/ssigy)**2)
    return exp(-sr**2)

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])

nx = 200
ny = 200
data = Array2d(-10.0,10.0,nx,-10.0,10.0,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = ConvolvedGaussian(data.x[i],data.y[j],sigmax=2.0, sigmay=1.0, c=2.0)
dataG = Array2d(-10.0,10.0,nx,-10.0,10.0,ny)
for i in range(nx):
    for j in range(ny):
        dataG.data[i,j] = Gaussian(data.x[i],data.y[j],sigmax=2.0, sigmay=1.0)
figure()
subplot(1,2,1)
imshow(data.data)
subplot(1,2,2)
plot(data.x,data.data[:,ny/2], color='red')
plot(data.y,data.data[nx/2,:], color='blue')
plot(data.x,dataG.data[:,ny/2], ls = '--',color='red')
plot(data.y,dataG.data[nx/2,:], ls = '--', color='blue')
ylim(0,1.2)
show()

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])

nx = 50
ny = 50
data = Array2d(-10.0,10.0,nx,-10.0,10.0,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = ConvolvedGaussian(data.x[i],data.y[j],sigmax=2.0, sigmay=1.0, c=2.0)
dataG = Array2d(-10.0,10.0,nx,-10.0,10.0,ny)
for i in range(nx):
    for j in range(ny):
        dataG.data[i,j] = ConvolvedGaussianFull(data.x[i],data.y[j],sigmax=2.0, sigmay=1.0, c=2.0)
NormFactor = ConvolvedGaussianFull(0.0,0.0,sigmax=2.0, sigmay=1.0, c=2.0)
print "NormFactor = %f"%NormFactor
#print dataG.data[:,ny/2]/NormFactor
figure()
subplot(1,3,1)
imshow(data.data)
subplot(1,3,2)
imshow(dataG.data)
subplot(1,3,3)
plot(data.x,data.data[:,ny/2], color='red')
plot(data.y,data.data[nx/2,:], color='blue')
plot(data.x,dataG.data[:,ny/2]/NormFactor, ls = '--',color='red')
plot(data.y,dataG.data[nx/2,:]/NormFactor, ls = '--', color='blue')
ylim(0,1.2)
show()

In [ ]:
# This seems like it works  Now how do I integrate it? Let me play with numerical schemes

# My guess on the 2D form doesn't match the numerically convolved form. 

In [ ]:
def ICG(xl, xh, yl, yh, sigmax, sigmay, c):
    NormFactor = 1.0#2.0*pi*sigmax*sigmay*(c*c+0.5) + 2.0*sigmax*sigmay*sqrt(pi)*c/erf(c)*exp(-c*c)
    nx = 100
    ny = 100
    data = Array2d(xl,xh,nx,yl,yh,ny)
    sum = 0.0
    for i in range(nx):
        for j in range(ny):
            data.data[i,j] = ConvolvedGaussian(data.x[i],data.y[j],sigmax=sigmax, sigmay=sigmay, c=c)
            sum += data.data[i,j] * data.dx * data.dy
    return sum / NormFactor

def IG(xl, xh, yl, yh, sigmax, sigmay):
    nx = 20
    ny = 20
    data = Array2d(xl,xh,nx,yl,yh,ny)
    sum = 0.0
    for i in range(nx):
        for j in range(ny):
            data.data[i,j] = Gaussian(data.x[i],data.y[j],sigmax=sigmax, sigmay=sigmay)
            #print i,j,data.data[i,j],sum
            sum += data.data[i,j] * data.dx * data.dy
    return sum / (2*pi)

def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
nx = 9
ny = 9
sigmax = 1.2
sigmay = 1.05
offx = 0.2
offy = -0.3
c=0.8
data = Array2d(-4.5,4.5,nx,-4.5,4.5,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = ICG(data.x[i]-data.dx/2+offx,data.x[i]+data.dx/2+offx,data.y[j]-data.dy/2+offy,data.y[j]+data.dy/2+offy,sigmax=sigmax, sigmay=sigmay, c=c)
dataG = Array2d(-4.5,4.5,nx,-4.5,4.5,ny)
for i in range(nx):
    for j in range(ny):
        dataG.data[i,j] = IG(data.x[i]-data.dx/2+offx,data.x[i]+data.dx/2+offx,data.y[j]-data.dy/2+offy,data.y[j]+data.dy/2+offy,sigmax=sigmax, sigmay=sigmay)
dataGA = Array2d(-4.5,4.5,nx,-4.5,4.5,ny)
for i in range(nx):
    for j in range(ny):
        dataGA.data[i,j] = Area(data.x[i]-data.dx/2+offx,data.x[i]+data.dx/2+offx,data.y[j]-data.dy/2+offy,data.y[j]+data.dy/2+offy,sigmax, sigmay, 1.0)
figure()
subplot(2,3,1)
imshow(data.data, interpolation='nearest')
subplot(2,3,2)
imshow(dataG.data, interpolation='nearest')
subplot(2,3,3)
imshow(dataGA.data, interpolation='nearest')
subplot(2,3,4)
plot(data.x,data.data[:,4], color='red')
plot(data.y,data.data[4,:], color='blue')
subplot(2,3,5)
plot(data.x,dataG.data[:,4], color='red')
plot(data.y,dataG.data[4,:], color='blue')
subplot(2,3,6)
plot(data.x,dataGA.data[:,4], color='red')
plot(data.y,dataGA.data[4,:], color='blue')
show()
#print abs(dataGA.data - dataG.data)/dataGA.data

In [ ]:
Int = IG(-10.0,10.0,-10.0,10.0,1.0,1.0)
print "Error=%f"%(abs(Int-1.0))
Int = Area(-10.0,10.0,-10.0,10.0,1.0,1.0,1.0)
print "Error=%f"%(abs(Int-1.0))
Int = ICG(-10.0,10.0,-10.0,10.0,1.0,1.0,2.0)
print "Error=%f"%(abs(Int-1.0))

In [ ]:
cs = linspace(0,4.0,100)
cs=delete(cs,cs[0])
c2s=[]
ints = []
sigmax = 1.0
sigmay = 1.0
for c in cs:
    int = ICG(-20.0,20.0,-20.0,20.0,sigmax, sigmay,c)
    #print c, int
    ints.append(int)
    c2s.append(c*c)
c2s=array(c2s)
ints=array(ints)
plot(c2s, ints)
#rom scipy import stats
#lope, intercept, r_value, p_value, std_err = stats.linregress(c2s[10:],ints[10:])
#rint "Intercept = %f, Slope = %f"%(intercept,slope)
#plot = pi + 2*pi * c2s + pi * exp(-c2s/sqrt(2.0))
yplot = 2*pi*sigmax**2*(c2s+0.5) + 2*sigmax**2*sqrt(pi)*cs/erf(cs)*exp(-c2s)
error = abs(yplot-ints)
print error.max()
plot(c2s, yplot, color='red', ls='--')
show()

In [ ]:
# OK, this clearly works in the case of sigmax = sigmay.  What about when they aren't equal?

In [ ]:
cs = linspace(0,4.0,100)
cs=delete(cs,cs[0])
c2s=[]
ints = []
sigmax = 1.0
sigmay = 0.50
sigma2 = 2.0*sigmax*sigmay
for c in cs:
    int = ICG(-20.0,20.0,-20.0,20.0,sigmax, sigmay,c)
    #print c, int
    ints.append(int)
    c2s.append(c*c)
c2s=array(c2s)
ints=array(ints)
plot(c2s, ints)
yplot = pi*sigma2*(c2s+0.5) + sigma2*sqrt(pi)*cs/erf(cs)*exp(-c2s)
#print yplot/ints
error = abs(yplot-ints)
print error.max()
plot(c2s, yplot, color='red', ls='--')
show()

In [ ]:
def ConvolvedGaussian(x, y, sigmax=1.0, sigmay=1.0, c=0.5):
    ssigx = sigmax * sqrt(2.0)
    ssigy = sigmay * sqrt(2.0)
    sr = sqrt((x/ssigx)**2 + (y/ssigy)**2)
    return (erf(sr+c) - erf(sr-c)) / (erf(c) - erf(-c))

def ICG(xl, xh, yl, yh, sigmax, sigmay, c):
    # This is normalized to have an Integral of 1.0
    NormFactor = 2.0*pi*sigmax*sigmay*(c*c+0.5) + 2.0*sigmax*sigmay*sqrt(pi)*c/erf(c)*exp(-c*c)
    nx = 20
    ny = 20
    data = Array2d(xl,xh,nx,yl,yh,ny)
    sum = 0.0
    for i in range(nx):
        for j in range(ny):
            data.data[i,j] = ConvolvedGaussian(data.x[i],data.y[j],sigmax=sigmax, sigmay=sigmay, c=c)
            sum += data.data[i,j] * data.dx * data.dy
    return sum / NormFactor


In [ ]:
def CGIntegrand(y, xh, yh, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xh
    ym = yc - xh
    f = yp * erf(yp / ssx) - ym * erf(ym / ssx)
    f += ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) / sqrt(pi)
    f *= erf((y+yh) / ssy)
    return f

def ICGStep(xh, yh, sigmax, sigmay, c):
    ny = 100
    dy = 2 * c / (ny - 1)
    sum = 0.0
    y = -c + dy / 2.0
    for i in range(ny-1):
        sum += dy * CGIntegrand(y,xh,yh,sigmax,sigmay,c)
        y += dy
    return sum

def ICGFull(xl, xh, yl, yh, sigmax, sigmay, c):
    sum = ICGStep(xh,yh,sigmax,sigmay,c) + ICGStep(xl,yl,sigmax,sigmay,c)
    sum -= ICGStep(xh,yl,sigmax,sigmay,c) + ICGStep(xl,yh,sigmax,sigmay,c)
    return sum
        

In [ ]:
xs = linspace(-8.0,8.0,100)
ys = ICGStep(xs,xs,1.0,1.0,2.0)
plot(xs,ys)
show()
print ICGStep(8.0,8.0,1.0,1.0,2.0)

In [ ]:
nx = 9
ny = 9
sigmax = 0.75
sigmay = 0.75
offx = 0.2
offy = -0.3
c=0.2
NormFactor = ICGFull(-10.0,10.0,-10.0,10.0,sigmax, sigmay, c)
data = Array2d(-4.5,4.5,nx,-4.5,4.5,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = ICGFull(data.x[i]-data.dx/2+offx,data.x[i]+data.dx/2+offx,data.y[j]-data.dy/2+offy,data.y[j]+data.dy/2+offy,sigmax=sigmax, sigmay=sigmay, c=c) / NormFactor
dataG = Array2d(-4.5,4.5,nx,-4.5,4.5,ny)
for i in range(nx):
    for j in range(ny):
        dataG.data[i,j] = IG(data.x[i]-data.dx/2+offx,data.x[i]+data.dx/2+offx,data.y[j]-data.dy/2+offy,data.y[j]+data.dy/2+offy,sigmax=sigmax, sigmay=sigmay)
dataGA = Array2d(-4.5,4.5,nx,-4.5,4.5,ny)
for i in range(nx):
    for j in range(ny):
        dataGA.data[i,j] = Area(data.x[i]-data.dx/2+offx,data.x[i]+data.dx/2+offx,data.y[j]-data.dy/2+offy,data.y[j]+data.dy/2+offy,sigmax, sigmay, 1.0)
figure()
subplot(2,3,1)
imshow(data.data, interpolation='nearest')
subplot(2,3,2)
imshow(dataG.data, interpolation='nearest')
subplot(2,3,3)
imshow(dataGA.data, interpolation='nearest')
subplot(2,3,4)
plot(data.x,data.data[:,4], color='red')
plot(data.y,data.data[4,:], color='blue')
subplot(2,3,5)
plot(data.x,dataG.data[:,4], color='red')
plot(data.y,dataG.data[4,:], color='blue')
subplot(2,3,6)
plot(data.x,dataGA.data[:,4], color='red')
plot(data.y,dataGA.data[4,:], color='blue')
show()
#print abs(dataGA.data - dataG.data)/dataGA.data

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])


def CGSpotIntegrand(y1, x, y, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    f = sqrt(pi/2.0) * sigmax *(erf((sqrt(c*c-y1*y1)+x)/ssx) - erf((x-sqrt(c*c-y1*y1))/ssx))
    f *= exp(-(y-y1)*(y-y1) / (ssy*ssy))
    return f

def CGFunc(x, y, sigmax, sigmay, c):
    return quad(CGSpotIntegrand, -c, c, args=(x,y,sigmax,sigmay,c))[0]
sigmax=1.0
sigmay=2.0
c=2.2
#print CGFunc(-4.0,0,sigmax,sigmay,c)
#sys.exit()
NormFactor = CGFunc(0.0,0.0,sigmax,sigmay,c)
nx = 100
ny = 100
data = Array2d(-10.0,10.0,nx,-10.0,10.0,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = CGFunc(data.x[i],data.y[j],sigmax, sigmay, c) / NormFactor
figure()
subplot(1,2,1)
imshow(data.data)
subplot(1,2,2)
plot(data.x,data.data[:,ny/2], color='red')
plot(data.y,data.data[nx/2,:], color='blue')
ylim(0,1.2)
show()

In [ ]:
x=linspace(0, 10.0,200)
y = exp(-(x/sigmax)**5.0)
plot(x,y)
ylim(0,1.2)
show()

In [ ]:
class Array2d:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        self.nx=nx
        self.ny=ny

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny])


def CGFuncTry(x, y, sigmax, sigmay, c):
    f = erf(x/(sqrt(2)*sigmax)+c)-erf(x/(sqrt(2)*sigmax)-c)
    f = f * (erf(y/(sqrt(2)*sigmay)+c)-erf(y/(sqrt(2)*sigmay)-c))
    return f
sigmax=1.0
sigmay=2.0
c=1.8
print CGFuncTry(0.0,0,sigmax,sigmay,c)
#sys.exit()
NormFactor = CGFuncTry(0.0,0.0,sigmax,sigmay,c)
nx = 100
ny = 100
data = Array2d(-10.0,10.0,nx,-10.0,10.0,ny)
for i in range(nx):
    for j in range(ny):
        data.data[i,j] = CGFuncTry(data.x[i],data.y[j],sigmax, sigmay, c) / NormFactor
figure()
subplot(1,2,1)
imshow(data.data)
subplot(1,2,2)
plot(data.x,data.data[:,ny/2], color='red')
plot(data.y,data.data[nx/2,:], color='blue')
ylim(0,1.2)
show()

In [ ]:
erf(100.0)

In [ ]:
from pylab import *
from scipy.special import erf
from scipy.integrate import quad

sigmax=1.0
sigmay=1.0
c=1.8


def CGIntegrand(y, xl, yl, sigmax, sigmay, c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    yp = yc + xl
    ym = yc - xl
    f = yp * erf(yp / ssx) - ym * erf(ym / ssx)
    f += ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) / sqrt(pi)
    f *= erf((y+yl) / ssy) - 1.0
    f -= 2.0 * yc * erf((y+yl) / ssy)
    f += pi * c / 2.0
    return f / (4.0 * pi * c * c)

def CGI2(y, l, sigma, c):
    ss = sigma * sqrt(2.0)
    yc = sqrt(c*c - y*y)
    f = 2.0 * yc * erf((y+l) / ss)
    return f

def CGI3(y, l, sigma, c):
    f = pi * c / 2.0
    return f

def ICGFull(xl, yl, sigmax, sigmay, c):
    return quad(CGIntegrand, -c, c, args=(xl,yl,sigmax,sigmay,c))[0]

def Term(xl,sigmax,sigmay,c):
    ssx = sigmax * sqrt(2.0)
    ssy = sigmay * sqrt(2.0)
    yp = c + xl
    ym = c - xl
    f = yp * erf(yp / ssx) - ym * erf(ym / ssx)
    f += ssx * (exp(-yp * yp / (ssx * ssx)) - exp(-ym * ym / (ssx * ssx))) * sqrt(2.0) / sqrt(pi)
    return f


        

In [ ]:
print quad(CGIntegrand, -c, c, args=(10.0,10.0,sigmax, sigmay,c))[0]
print pi * c * c
print quad(CGI3, -c, c, args=(1.0,sigmax,c))[0]
print quad(CGIntegrand, -c, c, args=(1.0,10.0,sigmax,sigmay,c))[0]
print quad(CGI2, -c, c, args=(1.0,sigmay,c))[0]
print quad(CGIntegrand, -c, c, args=(10.0,1.0,sigmax,sigmay,c))[0]

In [ ]:
print ICGFull(-100.0,-100.0, sigmax, sigmay, c)
print ICGFull(100.0,100.0, sigmax, sigmay, c)
print ICGFull(1.0,100.0, sigmax, sigmay, c)
print ICGFull(100.0,1.0, sigmax, sigmay, c)
print ICGFull(0.0,0.0, sigmax, sigmay, c)
print ICGFull(1.0,2.0, sigmax, sigmay, c)
print ICGFull(2.0,1.0, sigmax, sigmay, c)

In [ ]:
print CGIntegrand(1.0, 100000.0, 0.0, sigmax, sigmay, c)
print 2.0 * sqrt(c*c-1.0)* erf((1.0) / (sigmay * sqrt(2.0)))
print 4.0 * c * c
print quad(CGIntegrand, -c, c, args=(100.0,100.0,sigmax,sigmay,c))[0]

In [ ]:
xs = linspace(-8.0,8.0,100)
ys = []
for x in xs:
    #ys.append(quad(CGIntegrand, -c, c, args=(x,100.0,sigmax,sigmay,c))[0])
    ys.append(ICGFull(x,x,sigmax, sigmay, c))
plot(xs,ys)
#ys = []
#for x in xs:
#    #ys.append(quad(CGIntegrand, -c, c, args=(x,100.0,sigmax,sigmay,c))[0])
#    ys.append(ICGFull(x,20.0,1.0,1.0,2.0))
#plot(xs,ys)
show()
